In [1]:
ref = '/mnt/d/osm2lulc02/lmt/rst_terceira.tif'

osm_db = 'osm_terceira'

nomenclature = 'clc'

In [2]:
from glass.ete.osm2lulc.utilsv15 import nomenclature_id, osmfeat_by_rule
from glass.pys.oss import fprop
from glass.cons.osmtolulc import OSM2LULC_DB, DB_SCHEMA, OSM_TABLES
from glass.sql.q import q_to_obj

In [8]:
# Check if Nomenclature is valid
nom_id = nomenclature_id(nomenclature)

keycol = DB_SCHEMA["OSM_FEATURES"]["KEY"]
valcol = DB_SCHEMA["OSM_FEATURES"]["VALUE"]
lulcol = DB_SCHEMA['OSM_LULC']['LULCID']
modcol = DB_SCHEMA['MODULES']['NAME']

df = osmfeat_by_rule(nom_id)

df.loc[:, valcol] = df[keycol] + "='" + df[valcol] + "'"

# Get modules
mods = df[modcol].unique()

In [ ]:
# For each module
# Create a query to select features related to each module

for mod in mods:
    # Get keys and values of each Module
    fdf = df[df[modcol] == mod]

    # Get Database table to be used
    otbl = 'lines' if mod == 'roads' or mod == 'basic_buffer' \
        else 'polygons'

    bfcol = DB_SCHEMA['OSM_LULC']['BUFFER'] if mod == 'roads' \
        or mod == 'basic_buffer' else None
        
    arcol = DB_SCHEMA['OSM_LULC']['AREA'] if mod == 'area_upper' \
        or mod == 'area_lower' else None
    
    # Get classes inside this module
    lulcs = fdf[lulcol].unique()

    # For each LULC Class, Get a query to select features to be related with that class
    for cls in lulcs:
        # Get only the keys and values of this class
        clsdf = fdf[fdf[lulcol] == cls]

        # Create a query to select these keys and values
        q = (
            f"SELECT '{mod}' AS module, {cls} AS lulc, "
        )

In [10]:
df

,key,value,name,lulc_id,buffer_dist,area
0,landuse,landuse='allotments',selection,24,NaN,NaN
1,landuse,landuse='farmland',selection,24,NaN,NaN
2,landuse,landuse='farmyard',selection,24,NaN,NaN
3,landuse,landuse='farm',selection,24,NaN,NaN
4,natural,natural='mud',selection,26,NaN,NaN
...,...,...,...,...,...,...
520,tourism,tourism='gallery',selection,103,NaN,NaN
521,tourism,tourism='museum',selection,103,NaN,NaN
522,tourism,tourism='picnic_site',selection,103,NaN,NaN
523,tourism,tourism='theme_park',selection,103,NaN,NaN


In [9]:
mods

array(['selection', 'buildings', 'area_upper', 'basic_buffer', 'roads',
       'area_lower'], dtype=object)

In [ ]:
osm = '/mnt/disk1/jasp/autocls01/osmdata/osm_terceira.xml'

nomenclature = 'clc'

refraster = '/mnt/disk1/jasp/autocls01/ref/rst_terceira_3857.tif'

result = '/mnt/disk1/jasp/autocls01/lulc_fmosm/clc_terceira_v3.shp'

db = '/mnt/disk1/jasp/autocls01/lulc_fmosm/db_terceira_v3.sql'

In [ ]:
from glass.ete.osm2lulc.grs import osm_to_lulc
from glass.ete.osm2lulc.utils import record_time_consumed
import datetime
import os
from glass.pys.oss import fprop, mkdir
from glass.pys.tm import now_as_str

In [ ]:
time_a = datetime.datetime.now().replace(microsecond=0)

ws = mkdir(os.path.join(
    os.path.dirname(osm),
    fprop(result, 'fn')
), overwrite=True)

shp, time_check = osm_to_lulc(
    osm, nomenclature, refraster,
    result, tmpfld=ws,
    overwrite=True, savedb=db
)

time_b = datetime.datetime.now().replace(microsecond=0)
    
# Record time consumed in xlsx table
record_time_consumed(time_check, os.path.join(
    os.path.dirname(result),
    fprop(result, 'fn') + '.xlsx'
))

print(time_b - time_a)